# 从YouTube链接中加载文档

在YouTube视频上构建聊天或QA应用是一个备受关注的话题。

以下是如何轻松地从YouTube链接转换为文本到聊天的演示！

我们将使用 `OpenAIWhisperParser`，它将使用OpenAI Whisper API将音频转录为文本。

注：您需要提供一个 `OPENAI_API_KEY`。

In [1]:
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import OpenAIWhisperParser
from langchain.document_loaders.blob_loaders.youtube_audio import YoutubeAudioLoader

我们将使用`yt_dlp`下载YouTube链接的音频文件。

我们将使用`pydub`拆分下载的音频文件（以便我们遵守Whisper API的25MB文件大小限制）。

In [ ]:
! pip install yt_dlp
! pip install pydub

### 从YouTube链接到文本

使用`YoutubeAudioLoader`获取/下载音频文件。

然后，使用`OpenAIWhisperParser()`将其转录为文本。

让我们以Andrzej Karpathy的YouTube课程的第一讲为例！

In [2]:
# Two Karpathy lecture videos
urls = ["https://youtu.be/kCc8FmEb1nY",
        "https://youtu.be/VMj-3S1tku0"]

# Directory to save audio files 
save_dir = "~/Downloads/YouTube"

# Transcribe the videos to text
loader = GenericLoader(YoutubeAudioLoader(urls,save_dir),OpenAIWhisperParser())
docs = loader.load()

[youtube] Extracting URL: https://youtu.be/kCc8FmEb1nY
[youtube] kCc8FmEb1nY: Downloading webpage
[youtube] kCc8FmEb1nY: Downloading android player API JSON
[info] kCc8FmEb1nY: Downloading 1 format(s): 140
[dashsegments] Total fragments: 11
[download] Destination: /Users/31treehaus/Desktop/AI/langchain-fork/docs/modules/indexes/document_loaders/examples/Let's build GPT： from scratch, in code, spelled out..m4a
[download] 100% of  107.73MiB in 00:00:18 at 5.92MiB/s                   
[FixupM4a] Correcting container of "/Users/31treehaus/Desktop/AI/langchain-fork/docs/modules/indexes/document_loaders/examples/Let's build GPT： from scratch, in code, spelled out..m4a"
[ExtractAudio] Not converting audio /Users/31treehaus/Desktop/AI/langchain-fork/docs/modules/indexes/document_loaders/examples/Let's build GPT： from scratch, in code, spelled out..m4a; file is already in target format m4a
[youtube] Extracting URL: https://youtu.be/VMj-3S1tku0
[youtube] VMj-3S1tku0: Downloading webpage
[youtube

In [3]:
# Returns a list of Documents, which can be easily viewed or parsed
docs[0].page_content[0:500]

"Hello, my name is Andrej and I've been training deep neural networks for a bit more than a decade. And in this lecture I'd like to show you what neural network training looks like under the hood. So in particular we are going to start with a blank Jupyter notebook and by the end of this lecture we will define and train a neural net and you'll get to see everything that goes on under the hood and exactly sort of how that works on an intuitive level. Now specifically what I would like to do is I w"

### 从YouTube视频构建聊天应用

给定`文档`，我们可以轻松启用聊天/问答。

In [4]:
from langchain.chains import RetrievalQA
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [5]:
# Combine doc
combined_docs = [doc.page_content for doc in docs]
text = " ".join(combined_docs)

In [6]:
# Split them
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1500, chunk_overlap = 150)
splits = text_splitter.split_text(text)

In [7]:
# Build an index
embeddings = OpenAIEmbeddings()
vectordb = FAISS.from_texts(splits,embeddings)

In [8]:
# Build a QA chain
qa_chain = RetrievalQA.from_chain_type(llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0),
                                       chain_type="stuff",
                                       retriever=vectordb.as_retriever())

In [9]:
# Ask a question!
query = "Why do we need to zero out the gradient before backprop at each step?"
qa_chain.run(query)

"We need to zero out the gradient before backprop at each step because the backward pass accumulates gradients in the grad attribute of each parameter. If we don't reset the grad to zero before each backward pass, the gradients will accumulate and add up, leading to incorrect updates and slower convergence. By resetting the grad to zero before each backward pass, we ensure that the gradients are calculated correctly and that the optimization process works as intended."

In [10]:
query = "What is the difference between an encoder and decoder?"
qa_chain.run(query)

'In the context of transformers, an encoder is a component that reads in a sequence of input tokens and generates a sequence of hidden representations. On the other hand, a decoder is a component that takes in a sequence of hidden representations and generates a sequence of output tokens. The main difference between the two is that the encoder is used to encode the input sequence into a fixed-length representation, while the decoder is used to decode the fixed-length representation into an output sequence. In machine translation, for example, the encoder reads in the source language sentence and generates a fixed-length representation, which is then used by the decoder to generate the target language sentence.'

In [11]:
query = "For any token, what are x, k, v, and q?"
qa_chain.run(query)

'For any token, x is the input vector that contains the private information of that token, k and q are the key and query vectors respectively, which are produced by forwarding linear modules on x, and v is the vector that is calculated by propagating the same linear module on x again. The key vector represents what the token contains, and the query vector represents what the token is looking for. The vector v is the information that the token will communicate to other tokens if it finds them interesting, and it gets aggregated for the purposes of the self-attention mechanism.'